In [1]:
import sys
import lucene
 
from java.io import File
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene import document
from org.apache.lucene.document import Document, Field
from org.apache.lucene.index import IndexWriter, IndexWriterConfig, IndexOptions
from org.apache.lucene.search.similarities import TFIDFSimilarity
from org.apache.lucene.store import FSDirectory
from org.apache.lucene.util import Version
from helper import get_docs

INDEXER

In [2]:
lucene.initVM()

In [3]:
indexPath = File("index/").toPath() #from java. io import File
indexDir = FSDirectory.open(indexPath)
writerConfig = IndexWriterConfig(StandardAnalyzer())
writer = IndexWriter(indexDir, writerConfig)

In [4]:
docIdToText = get_docs("time/test.all")

In [5]:
print(docIdToText)

defaultdict(<class 'list'>, {1: ['The', 'dog', 'barked'], 2: ['The', 'dog', 'jumped'], 3: ['A', 'cat', 'jumped']})


In [6]:
for k, v in docIdToText.items():
    doc = Document()
    fieldType = document.FieldType()
    fieldType.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS_AND_OFFSETS)
    fieldType.setStored(True)
    fieldType.setTokenized(True)
    fieldType.setOmitNorms(False)

    doc.add(Field("myID", k, fieldType))
    doc.add(Field("content", "".join(v), fieldType))

    writer.addDocument(doc)


In [7]:
writer.close()

RETRIEVER

In [10]:
import sys
import lucene

from java.io import File
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene import document
from org.apache.lucene.document import Document, Field
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.search import similarities

from org.apache.lucene.search.similarities import TFIDFSimilarity
from org.apache.lucene.search.similarities import BM25Similarity
from org.apache.lucene.search.similarities import BooleanSimilarity
from org.apache.lucene.search.similarities import ClassicSimilarity
from org.apache.lucene.search.similarities import Similarity


from org.apache.lucene.index import IndexReader, DirectoryReader
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.store import FSDirectory
from org.apache.lucene.util import Version

In [11]:
lucene.initVM()

ValueError: JVM is already running and updating its classpath failed. Call initVM() instead just once but with a classpath keyword argument set to the module.CLASSPATH strings of all the JCC extension modules to be imported by this process

In [13]:
analyzer = StandardAnalyzer()
indexPath = File("index").toPath()
indexDir = FSDirectory.open(indexPath)
reader = DirectoryReader.open(indexDir)

JavaError: <super: <class 'JavaError'>, <JavaError object>>
    Java stacktrace:
org.apache.lucene.index.IndexNotFoundException: no segments* file found in MMapDirectory@/root/5180GroupAssignment1/Group_Assignment_3/index lockFactory=org.apache.lucene.store.NativeFSLockFactory@5d018107: files: [_0.fdm, _0.fdt, _0_Lucene90FieldsIndex-doc_ids_0.tmp, _0_Lucene90FieldsIndexfile_pointers_1.tmp, write.lock]
	at org.apache.lucene.index.SegmentInfos$FindSegmentsFile.run(SegmentInfos.java:803)
	at org.apache.lucene.index.StandardDirectoryReader.open(StandardDirectoryReader.java:109)
	at org.apache.lucene.index.StandardDirectoryReader.open(StandardDirectoryReader.java:67)
	at org.apache.lucene.index.DirectoryReader.open(DirectoryReader.java:60)


In [ ]:
print(reader)

NameError: name 'reader' is not defined

In [ ]:
print(reader.numDocs())

3


In [ ]:
searcher = IndexSearcher(reader)

In [ ]:
print(searcher)

IndexSearcher(StandardDirectoryReader(segments_1:4 _0(9.1.0):c3:[diagnostics={lucene.version=9.1.0, java.vm.version=11.0.16+8-post-Debian-1deb11u1, java.version=11.0.16, timestamp=1667844396571, os=Linux, java.vendor=Debian, os.version=5.15.49-linuxkit, java.runtime.version=11.0.16+8-post-Debian-1deb11u1, os.arch=aarch64, source=flush}]:[attributes={Lucene90StoredFieldsFormat.mode=BEST_SPEED}] :id=8nmfbz6pa20cv1xxohdd31iic); executor=null; sliceExecutionControlPlane null)


In [ ]:
searcher.similarity = ClassicSimilarity()

In [ ]:
print(searcher)

IndexSearcher(StandardDirectoryReader(segments_1:4 _0(9.1.0):c3:[diagnostics={lucene.version=9.1.0, java.vm.version=11.0.16+8-post-Debian-1deb11u1, java.version=11.0.16, timestamp=1667844396571, os=Linux, java.vendor=Debian, os.version=5.15.49-linuxkit, java.runtime.version=11.0.16+8-post-Debian-1deb11u1, os.arch=aarch64, source=flush}]:[attributes={Lucene90StoredFieldsFormat.mode=BEST_SPEED}] :id=8nmfbz6pa20cv1xxohdd31iic); executor=null; sliceExecutionControlPlane null)


In [ ]:
query = QueryParser("text", analyzer).parse("a cat jumped")

In [ ]:
print(query)

text:a text:cat text:jumped


In [ ]:
MAX = 1000
hits = searcher.search(query, MAX)

In [ ]:
print(hits)

org.apache.lucene.search.TopDocs@530a8454


In [ ]:
print(len(hits.scoreDocs))
        

0
